In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**First of all let's clone the longformer repository and nevigate to its directory**

In [ ]:
!git clone https://github.com/allenai/longformer.git

Cloning into 'longformer'...
remote: Enumerating objects: 1240, done.
remote: Counting objects: 100% (215/215), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1240 (delta 203), reused 198 (delta 198), pack-reused 1025
Receiving objects: 100% (1240/1240), 837.38 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (838/838), done.


In [ ]:
%cd longformer

/content/longformer


**Now we will install require packages**

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

**Lets import all the necessary libraries**

In [ ]:
import torch
from transformers import LongformerTokenizer, LongformerModel

**loading pretrained longformer**

In [ ]:
model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

**now we will prepare input and run the model**

In [ ]:
text = "This is an example of a very long document that we want to process using Longformer."
inputs = tokenizer(text, return_tensors='pt', max_length=4096, truncation=True, padding='max_length')


In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

# Retrieve the last hidden state
last_hidden_states = outputs.last_hidden_state

In [ ]:
# Print last hidden state details
print("Last Hidden States Shape:", last_hidden_states.shape)
print("Last Hidden States:", last_hidden_states)

Last Hidden States Shape: torch.Size([1, 4096, 768])
Last Hidden States: tensor([[[-0.0470,  0.1103, -0.0197,  ..., -0.1658,  0.0115, -0.0037],
         [-0.0020,  0.1571,  0.1532,  ...,  0.0158,  0.1495,  0.2231],
         [ 0.1668,  0.3675,  0.1867,  ..., -0.3038,  0.1431,  0.3824],
         ...,
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745]]])


In [ ]:
print("Tokenized Input IDs:", inputs['input_ids'])
print("Attention Mask:", inputs['attention_mask'])


Tokenized Input IDs: tensor([[  0, 713,  16,  ...,   1,   1,   1]])
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0]])


**Now lets create an upgraded version of the existing methodology.**

**Lets load a dataset for finetuning , here we have used IMDb dataset for sentiment analysis.**

In [ ]:
from datasets import load_dataset

dataset = load_dataset('imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Modify Model Hyperparameters**

In [ ]:
import torch
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments

model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

**Prepare Data for Training:**

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # Subset for quick experimentation
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

**Experiment with Hyperparameters ,We'll modify the learning rate and batch size as examples of hyperparameter changes.**

**Set Training Arguments:**

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,          # Modified learning rate
    per_device_train_batch_size=8,  # Modified batch size
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Initializing global attention on CLS token...


Epoch,Training Loss,Validation Loss
1,No log,0.285210
2,No log,0.361324
3,No log,0.480662


TrainOutput(global_step=375, training_loss=0.3153460896809896, metrics={'train_runtime': 730.4581, 'train_samples_per_second': 4.107, 'train_steps_per_second': 0.513, 'total_flos': 985277417472000.0, 'train_loss': 0.3153460896809896, 'epoch': 3.0})

**Evaluate on the Test Set:**

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.48066169023513794, 'eval_runtime': 26.0744, 'eval_samples_per_second': 19.176, 'eval_steps_per_second': 2.416, 'epoch': 3.0}


**Let's start by setting up a baseline model with default parameters and then we will change each parameter to see the differences.**

In [ ]:
from transformers import TrainingArguments, Trainer

# Set up baseline training arguments
baseline_training_args = TrainingArguments(
    output_dir="./results_baseline",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer with baseline settings
baseline_trainer = Trainer(
    model=model,
    args=baseline_training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the baseline model
baseline_trainer.train()

# Evaluate the baseline model
baseline_eval_results = baseline_trainer.evaluate()
print("Baseline Evaluation Results:", baseline_eval_results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.554120
2,No log,0.522093


Epoch,Training Loss,Validation Loss
1,No log,0.554120
2,No log,0.522093
3,No log,0.597216


Baseline Evaluation Results: {'eval_loss': 0.5972161293029785, 'eval_runtime': 25.9862, 'eval_samples_per_second': 19.241, 'eval_steps_per_second': 2.424, 'epoch': 3.0}


**Now We'll test three different learning rates: 1e-5, 2e-5, and 5e-5.**

In [ ]:
learning_rates = [1e-5, 2e-5, 5e-5]

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    training_args = TrainingArguments(
        output_dir=f"./results_lr_{lr}",
        evaluation_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Train the model with the specified learning rate
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation Results for learning rate {lr}:", eval_results)

Training with learning rate: 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.613347
2,No log,0.753174
3,No log,0.633946


Evaluation Results for learning rate 1e-05: {'eval_loss': 0.6339464783668518, 'eval_runtime': 26.0984, 'eval_samples_per_second': 19.158, 'eval_steps_per_second': 2.414, 'epoch': 3.0}
Training with learning rate: 2e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.680033
2,No log,0.812165
3,No log,0.692521


Evaluation Results for learning rate 2e-05: {'eval_loss': 0.6925205588340759, 'eval_runtime': 26.1084, 'eval_samples_per_second': 19.151, 'eval_steps_per_second': 2.413, 'epoch': 3.0}
Training with learning rate: 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.914951
2,No log,0.791469
3,No log,0.751127


Evaluation Results for learning rate 5e-05: {'eval_loss': 0.751127302646637, 'eval_runtime': 26.0991, 'eval_samples_per_second': 19.158, 'eval_steps_per_second': 2.414, 'epoch': 3.0}


**Also Let's experiment with batch sizes of 4**

In [ ]:
# Experiment with Batch Size 4
batch_size = 4
print(f"Training with batch size: {batch_size}")

training_args_bs4 = TrainingArguments(
    output_dir=f"./results_bs_{batch_size}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer_bs4 = Trainer(
    model=model,
    args=training_args_bs4,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model with batch size 4
trainer_bs4.train()

# Evaluate the model
eval_results_bs4 = trainer_bs4.evaluate()
print(f"Evaluation Results for batch size {batch_size}:", eval_results_bs4)


Training with batch size: 4


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.269984
2,0.047400,1.193018
3,0.047400,1.204575


Evaluation Results for batch size 4: {'eval_loss': 1.204574704170227, 'eval_runtime': 28.2237, 'eval_samples_per_second': 17.716, 'eval_steps_per_second': 4.429, 'epoch': 3.0}


**now as a final experiment We will try training the model for 2,and 4 epochs.**

In [ ]:
epochs = [2,4]

for epoch in epochs:
    print(f"Training for {epoch} epochs")

    training_args = TrainingArguments(
        output_dir=f"./results_epochs_{epoch}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=epoch,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    # Train the model with the specified number of epochs
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(f"Evaluation Results for {epoch} epochs:", eval_results)


Training for 2 epochs


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.165896
2,No log,1.186653


Evaluation Results for 2 epochs: {'eval_loss': 1.1866534948349, 'eval_runtime': 25.9843, 'eval_samples_per_second': 19.242, 'eval_steps_per_second': 2.425, 'epoch': 2.0}
Training for 4 epochs


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.344559
2,No log,1.605467
3,No log,1.125295
4,0.014700,0.987442


Evaluation Results for 4 epochs: {'eval_loss': 0.987441897392273, 'eval_runtime': 26.1034, 'eval_samples_per_second': 19.155, 'eval_steps_per_second': 2.413, 'epoch': 4.0}
